In [1]:
from selenium import webdriver
from datetime import datetime
import csv

In [10]:
#Path to Chrome Driver for windows
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

#Open site
driver.get("https://stockx.com/sneakers")

#Click on the annoying pop-up
driver.implicitly_wait(10)
PopUpQuitButton = driver.find_element_by_xpath("//img[@src='//stockx-assets.imgix.net/AllInAsk/close (1).svg?auto=compress,format']").click()

In [5]:
#Function that takes in date array and price array and returns the average price for the sale history of previous months
def get_avg_new_price(date_arr, price_arr):
    #Array for new prices
    new_price_arr = []

    #Find timestamp for last 2 month
    now = datetime.now().timestamp()
    month_seconds = 2628288
    last_month_ts = now - (month_seconds*2)

    #Filter out and append new prices to the array
    for i in range(len(date_arr)):
        if date_arr[i] > last_month_ts:
            new_price_arr.append(price_arr[i])

    #Averaging all values in new_price_arr
    average_price = sum(new_price_arr)/len(new_price_arr)
    return round(average_price,2)

In [12]:
#For loop to loop through all 40 pairs on the first page
for i in range(1):
    
    #Click on the i + 1 sneaker
    xpathstr = "//div[@class='tile browse-tile updated']["+str(i+1)+"]"
    SneakerSelect = driver.find_element_by_xpath(xpathstr).click()

    #Click on button to view all sales of the sneaker
    driver.find_element_by_xpath("//a[@data-testid='product-viewallsales']").click()

    #Loop to press load more button to load full table. It will press load more button 10 times and then it will break out of loop. If there is not up to 10 button presses, it will automatically leave the loop. This should allow a little over 100 of the sale history.
    press_count = 0
    while True:
        try:
            load_more_button = driver.find_element_by_xpath("//button[@class='button button-block button-white']").click()
            press_count += 1
            if press_count == 10:
                break
        except:
            break
    
    #Find element for the sale history table and 
    table = driver.find_element_by_xpath("//table[@id='480']").text

    #Clears table info file to make sure it is empty
    with open("Tableinfo.txt", "r+") as f:
        f.truncate(0)

    #Open table file and overwrite the file with the new data
    with open ("Tableinfo.txt", "w") as f:
        f.write(table)

    #Extract price and date and append to arrays
    price_arr = []
    date_arr = []

    #Function to get info of date, price and size from tableinfo.txt
    def get_info():
        
        with open("Tableinfo.txt","r") as f:
            lines =f.readlines()
            
            for line in lines:
                
                if "$" in line:
                    LineList = line.split("$")

                    #Get price and store it into an array of integers
                    price = int(LineList[1].replace(',','').strip())
                    price_arr.append(price)

                    #Split first half of line to get the date and the size
                    date_and_size = LineList[0]
                    date_size_split_list = date_and_size.split("EST")

                    #Format date into timestamp and append to array
                    date_string = date_size_split_list[0]
                    formatted_date = extract_date(date_string)
                    date_arr.append(formatted_date)

    get_info()

    #Finding average for sales in previous month
    avg_price = get_avg_new_price(date_arr, price_arr)

    #Append average price to csv file
    with open('AveragePrice.csv', 'a') as csvfile:
        writer = csv.writer(csvfile, lineterminator = '\r')
        writer.writerow([avg_price])

    #Clear arrays and reset all counters, ready for the next pair
    price_arr.clear()
    date_arr.clear()

    #Return to initial page and refresh
    driver.back()
    driver.refresh()